In [1]:
import sys
import os
import numpy as np

# path to folder to CityGeoTools
folder = "/var/essdata/IDU/other/Rest_/refactored/CityGeoTools"
sys.path.append(folder)

from metrics.data import CityInformationModel as BaseModel

/var/essdata/IDU/venvs/common_venv/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


### Create CityInformationModel

To define own City Information Model, you need to create an instance of CityInformationModel class with specified attributes: 
   
**city_name** - string (name of city which you are going to work with)  
**city_crs** - integer (EPSG projection of city)   
**cwd** - relative path to CityGeoTools library 


In [2]:
city_model = BaseModel.CityInformationModel(city_name="Saint-Petersburg", city_crs=32636, cwd="../")
city_model

The object of class CityInformationModel contains attributes describing data required for successful method call.  
After creating own model these attributes equal to **None** by default.

In [3]:
city_model.get_all_attributes()

{'city_name': 'Saint-Petersburg',
 'city_crs': 32636,
 'city_id': typing.Optional[int],
 'cwd': '../',
 'mode': 'user_mode',
 'MobilityGraph': None,
 'Buildings': None,
 'Services': None,
 'PublicTransportStops': None,
 'ServiceTypes': None,
 'RecreationalAreas': None,
 'Blocks': None,
 'Municipalities': None,
 'AdministrativeUnits': None,
 'ValueTypes': None,
 'SocialGroups': None,
 'SocialGroupsValueTypesLivingSituations': None,
 'LivingSituationsCityServiceTypes': None,
 'LivingSituations': None,
 'methods': <metrics.data.DataValidation.DataValidation at 0x7fd10c6be040>}

### Update layers

To use CityGeoTools methods, it is necessary to load prepaid data to the model specifying attribute names and paths to files.  
Only **json**, **geojson** and **graphml** file formats are available for loading. Before data loading, all methods are marked as unavailable.  
Exception is weighted voronoi method, since it doesn't require any urban data.

In [4]:
print("All methods implemented in CityGeoTools:\n")

all_methods = city_model.methods.get_list_of_methods()
available_methods = city_model.methods.get_list_of_available_methods()

for method in all_methods:
    if method in available_methods:
        print(method, "--> available")
    else:
        print(method, "--> unavailable")


All methods implemented in CityGeoTools:

traffics_calculation --> unavailable
visibility_analysis --> unavailable
weighted_voronoi --> available
blocks_clusterization --> unavailable
services_clusterization --> unavailable
spacematrix --> unavailable
mobility_analysis --> unavailable
diversity --> unavailable
collocation_matrix --> unavailable
coverage_zones --> unavailable
master_plan --> unavailable
blocks_accessibility --> unavailable


In [5]:
# GeoJSON
city_model.update_layer("Buildings", "./data/buildings.geojson")
city_model.update_layer("Services", "./data/services.geojson")
city_model.update_layer("Blocks", "./data/blocks_with_wrong_type.geojson")

#JSON
city_model.update_layer("ServiceTypes", "./data/service_types.json")

#GraphML
city_model.update_layer("MobilityGraph", "./data/graph.graphml")

Validation of Buildings layer...
Buildings layer loaded successfully!
Validation of Services layer...
Services layer loaded successfully!
Validation of Blocks layer...
Blocks layer loaded successfully!
Validation of ServiceTypes layer...
ServiceTypes layer loaded successfully!
Validation of MobilityGraph layer...
MobilityGraph layer loaded successfully!


In [9]:
print("All methods implemented in CityGeoTools:\n")

all_methods = city_model.methods.get_list_of_methods()
available_methods = city_model.methods.get_list_of_available_methods()

for method in all_methods:
    if method in available_methods:
        print(method, "--> available")
    else:
        print(method, "--> unavailable")

All methods implemented in CityGeoTools:

traffics_calculation --> unavailable
visibility_analysis --> available
weighted_voronoi --> available
blocks_clusterization --> unavailable
services_clusterization --> unavailable
spacematrix --> unavailable
mobility_analysis --> unavailable
diversity --> unavailable
collocation_matrix --> unavailable
coverage_zones --> unavailable
master_plan --> available
blocks_accessibility --> unavailable


### Deal with ValidationError

It should be noted that all loaded data MUST match specification for certain method to successfully call it.  
Otherwise, you will encounter the ValidationError even though the file is saved into attribute.   
To know why loaded file does not match specification, see attribute **message** that is available trought city_model.methods  

In [10]:
from metrics.calculations import spacematrix
spacematrix.Spacematrix(city_model).get_morphotypes()

ValidationError: Layers Blocks do not match specification.

In [11]:
city_model.methods.spacematrix.message

{'Buildings': 'Layer matches specification',
 'Blocks': "79.0 is not of type 'integer'"}